In [1]:
!pip install faiss-cpu face_recognition

In [2]:
import face_recognition
import os
import numpy as np
import faiss
import pickle
from PIL import Image, ImageDraw

In [3]:
# 얼굴 DB 구축 및 Faiss 인덱스에 추가
# 얼굴 DB 구축 및 Faiss 인덱스에 추가
def build_face_db(image_dir):
    face_encodings = []
    face_labels = []
    face_locations = {}

    for image_file in os.listdir(image_dir):
        if image_file.endswith(('.jpg', '.jpeg', '.png')):
            # 이미지 파일 로드
            image_path = os.path.join(image_dir, image_file)
            image = face_recognition.load_image_file(image_path)
            # 모든 얼굴 인코딩과 얼굴 위치 추출
            encodings = face_recognition.face_encodings(image)
            locations = face_recognition.face_locations(image)

            if encodings:
                for i, (encoding, location) in enumerate(zip(encodings, locations)):
                    # 각 얼굴 인코딩을 저장
                    face_encodings.append(encoding)
                    # 레이블로 파일명과 얼굴 인덱스를 함께 저장 (ex. image_1_face_1.jpg)
                    label = f"{image_file}_face_{i}"
                    face_labels.append(label)
                    # 얼굴 위치도 함께 저장 (이미지 파일명, 얼굴 인덱스와 위치 매핑)
                    face_locations[label] = location

    # 얼굴 인코딩을 numpy 배열로 변환
    face_encodings_np = np.array(face_encodings)

    # 인코딩된 얼굴 벡터를 Faiss 인덱스에 추가
    index.add(face_encodings_np)

    # 얼굴 레이블 및 위치 저장
    with open('face_labels.pkl', 'wb') as f:
        pickle.dump(face_labels, f)

    with open('face_locations.pkl', 'wb') as f:
        pickle.dump(face_locations, f)

In [4]:
# 얼굴 이미지를 저장할 디렉토리 경로
IMAGE_DIR = './faces/'
# 벡터 차원 (face_recognition의 얼굴 인코딩 차원은 128)
VECTOR_DIM = 128

# Faiss 인덱스 생성 (L2 거리 사용)
index = faiss.IndexFlatL2(VECTOR_DIM)

In [5]:
# 얼굴 DB 구축 실행
build_face_db(IMAGE_DIR)

#### 비슷한 얼굴 쿼리하기 (모든 얼굴 중 유사한 얼굴 찾기)

In [6]:
# 유사한 얼굴 찾기 및 이미지에 얼굴 그리기
def find_similar_faces(query_image_path, index, face_labels, face_locations, k=5):
    # 쿼리 이미지 로드
    query_image = face_recognition.load_image_file(query_image_path)
    query_encodings = face_recognition.face_encodings(query_image)

    if not query_encodings:
        print("이미지에서 얼굴을 찾을 수 없습니다.")
        return []

    similar_faces = []
    # 쿼리 이미지의 모든 얼굴에 대해 검색
    for query_encoding in query_encodings:
        query_encoding = query_encoding.reshape(1, -1)

        # Faiss 인덱스에서 가장 가까운 K개의 얼굴 찾기
        distances, indices = index.search(query_encoding, k)

        # 결과 저장
        for i, idx in enumerate(indices[0]):
            similar_faces.append((face_labels[idx], distances[0][i]))

    # 거리 기준으로 정렬
    similar_faces = sorted(similar_faces, key=lambda x: x[1])

    return similar_faces

In [7]:
# 쿼리 이미지와 매칭된 얼굴을 박스 처리하여 그리기
def draw_faces(matched_faces, face_locations):
    for face, _ in matched_faces:
        # 얼굴 레이블에서 파일명과 얼굴 인덱스 추출
        image_file = face.split('_face_')[0]
        image_path = os.path.join(IMAGE_DIR, image_file)
        
        # 매칭된 DB 이미지 로드
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # 얼굴 위치 가져오기
        location = face_locations.get(face)
        if location:
            top, right, bottom, left = location
            # 얼굴에 박스 그리기
            draw.rectangle(((left, top), (right, bottom)), outline=(0, 255, 0), width=3)

        # 매칭된 얼굴 이미지 보여주기
        image.show()

In [8]:
# 얼굴 레이블 및 위치 로드
with open('face_labels.pkl', 'rb') as f:
    face_labels = pickle.load(f)

with open('face_locations.pkl', 'rb') as f:
    face_locations = pickle.load(f)

In [9]:
query_image_path = './test_hanni.jpg'

In [10]:
# 쿼리 이미지로 비슷한 얼굴 찾기
similar_faces = find_similar_faces(query_image_path, index, face_labels, face_locations, k=10)

if similar_faces:
    print("비슷한 얼굴들:")
    for name, distance in similar_faces:
        print(f"{name} (유사도 거리: {distance})")
    
    # 매칭된 이미지에서 얼굴을 그리기
    draw_faces(similar_faces, face_locations)
else:
    print("유사한 얼굴을 찾을 수 없습니다.")

비슷한 얼굴들:
newjeans02.jpg_face_4 (유사도 거리: 0.06112991273403168)
newjeans01.jpg_face_0 (유사도 거리: 0.09109141677618027)
hanni02.jpg_face_0 (유사도 거리: 0.1049562394618988)
hanni06.jpg_face_0 (유사도 거리: 0.1453244686126709)
newjeans02.jpg_face_0 (유사도 거리: 0.154688760638237)
